This has been optimized

In [8]:
import time
import os 

start_time = time.time()

WORKSPACE_DIR = os.getenv('WORKSPACE_DIR')
if not os.getcwd().endswith('portfolio_py'):
    os.chdir(f'{WORKSPACE_DIR}/portfolio_py')
print(f'Current Working Directory: {os.getcwd()}')

from utils.finance_utils import calculate_sharpe_ratio
from utils.helpers import divide_chunks

Current Working Directory: /Users/blakeuribe/Desktop/portfolio_py


In [9]:
from dotenv import load_dotenv

import yfinance as yf
import numpy as np
import pandas as pd

from datetime import datetime
from dateutil.relativedelta import relativedelta

In [10]:
# Load environment variables
load_dotenv()
TODAY = datetime.today()
ONE_YEAR_AGO = TODAY - relativedelta(years=1)

print(f'Today is: {TODAY}')
print(f'One Year Ago was: {ONE_YEAR_AGO}')

Today is: 2025-01-15 14:14:23.176267
One Year Ago was: 2024-01-15 14:14:23.176267


In [11]:
# Download and import data
ticker_df = pd.read_json(f'{WORKSPACE_DIR}/portfolio_py/data/raw/company_tickers.json')
ticker_df = ticker_df.T.drop(columns=['cik_str', 'title'])

tbill_data = yf.download('^IRX', start=ONE_YEAR_AGO, end=TODAY, auto_adjust=True)['Close']
tbill_data = tbill_data / 100 / 360  # Convert to daily rate

spy_sharpe = calculate_sharpe_ratio(np.array('spy'), tbill=tbill_data, start_date=ONE_YEAR_AGO, end_date=TODAY)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [14]:

# Set chunk size and number of stocks
num_in_chunks = 25
num_of_stocks_to_get = 1500  # Adjust as needed

# Get tickers and divide into chunks

print('----Dividing Into Chunks----')
tickers = ticker_df['ticker'][0:num_of_stocks_to_get]  # Remove .tolist() here
ticker_chunks = list(divide_chunks(tickers, num_in_chunks))

print(f'Divided {num_of_stocks_to_get} tickers into {len(ticker_chunks)} chunks')

# Initialize an empty list to store results
df_list = []

print('\n----Collecting Sharpe Ratios----')
# Loop through each chunk and process the stocks
for chunk in ticker_chunks:
    try:
        # Calculate Sharpe ratios for the current chunk
        sharpe_ratios_series = calculate_sharpe_ratio(
            chunk, 
            tbill=tbill_data, 
            start_date=ONE_YEAR_AGO, 
            end_date=TODAY
        )
        # Create a DataFrame for the chunk
        results_df_chunk = pd.DataFrame({
            'Tickers': sharpe_ratios_series.index, 
            'Sharpe_ratios': sharpe_ratios_series.values
        })
        df_list.append(results_df_chunk)

    except Exception as e:
        print(f'Error calculating Sharpe ratios for chunk: {chunk}\n{str(e)}')

# Concatenate all DataFrames vertically
final_df = pd.concat(df_list, axis=0, ignore_index=True)


# Ensure we have a benchmark value for futre use
# spy_sharpe = calculate_sharpe_ratio(np.array('spy'), tbill=tbill_data, start_date=ONE_YEAR_AGO, end_date=TODAY)
final_df.loc[len(final_df)] = ['SPY', float(spy_sharpe.values[0])]

final_df = final_df.drop_duplicates()

# Check percentage of NaN values
pct_of_nan = (final_df.isna().sum().sum() / len(final_df) * 100).round(2)
print('\n----Df Report----')
print(f'Pct of NaN values is: {pct_of_nan}%')

# Save the results to CSV
if pct_of_nan <= 25: # Checks to see if Df is worth exporting
    final_df.to_csv(f'{WORKSPACE_DIR}/portfolio_py/data/clean/sharpe_ratios.csv', index=False)
    print('Df contianing Sharpe Ratios Successfully Exported')
else:
    print('Error: Nan Values are too great to epxort Df')

end_time = time.time()
elapsed_time = end_time - start_time

print('\n----Time Report----')
print(f'Processing time: {elapsed_time:.2f} seconds, for {num_of_stocks_to_get} Tickers')

[**********            20%                       ]  5 of 25 completed

----Dividing Into Chunks----
Divided 1500 tickers into 60 chunks

----Collecting Sharpe Ratios----


[*********************100%***********************]  25 of 25 completed
[*********************100%***********************]  25 of 25 completed
[*********************100%***********************]  25 of 25 completed
[*********************100%***********************]  25 of 25 completed
[*********************100%***********************]  25 of 25 completed
/Users/blakeuribe/Desktop/portfolio_py/utils/finance_utils.py:16: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_returns = stock_data.pct_change()
[*********************100%***********************]  25 of 25 completed
[*********************100%***********************]  25 of 25 completed
[*********************100%***********************]  25 of 25 completed
[*********************100%***********************]  25 of 25 completed
[******


----Df Report----
Pct of NaN values is: 19.99%
Df contianing Sharpe Ratios Successfully Exported

----Time Report----
Processing time: 210.36 seconds, for 1500 Tickers


In [15]:
final_df

,Tickers,Sharpe_ratios
0,AAPL,1.253248
1,AMZN,1.533470
2,AVGO,1.828059
3,BAC,1.966664
4,BRK-B,1.622133
...,...,...
1496,RXO,NaN
1497,TIGO,NaN
1498,TXNM,NaN
1499,WRD,NaN
